In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# parameters (adjust as needed)
interval = 2            # Time between frames in seconds 
threshold_buffer = 0.002      # Threshold buffer above baseline 
window1 = None                # AUC window after threshold crossing – Peak 1 (s)
window2 = None                # AUC window – Peak 2 (s)

# Load Data and create time axis
df = pd.read_excel("FILENAME.xlsx")  # Replace with the actual filename
df.insert(0, "Time", np.arange(0, len(df) * interval, interval)) #insert time axis
time = df["Time"].to_numpy()
data = df.iloc[:, 1:].to_numpy()
num_cells = data.shape[1]

# Baseline intervals [s] (adjust as needed)
baseline1_start = None #start, where baseline for peak 1 will begin
baseline1_end = None   #end, where baseline for peak 1 will end
baseline2_start = None #start, where baseline for peak 2 will begin
baseline2_end = None   #end, where baseline for peak 2 will end



# Extracts the part of the signal between start and end time.
# Returns the average value (baseline) of this segment.
def get_baseline(trace, time, start, end): 
    idx_start = np.searchsorted(time, start)
    idx_end = np.searchsorted(time, end)
    return trace[idx_start:idx_end].mean()


# calculates AUC for each cell 
# looks for threshold crossing -> measurement starts in a fixed time window 
def analyze_auc_per_cell(data, time, baseline_start, baseline_end,   
                         search_start, search_end, window_sec, buffer):
    idx_search_start = np.searchsorted(time, search_start) # converts search start time in index
    idx_search_end = np.searchsorted(time, search_end) # converts search end time in index

    aucs = [] # initialize list for later storage of the results 
    start_times = []


# for each cell i, get it's trace and calculate baseline and threshold 
    for i in range(data.shape[1]):
        trace = data[:, i] # Extract the full time series for the i-th cell (all time points in column i)
        baseline = get_baseline(trace, time, baseline_start, baseline_end)
        threshold = baseline + buffer

        # get the part of the trace, where we look for the calcium response
        trace_segment = trace[idx_search_start:idx_search_end]
        time_segment = time[idx_search_start:idx_search_end]

        above = trace_segment > threshold # above has length of idx_search_end - idx_search_start
        if not np.any(above):             # if the cell never crosses the threshold, the cell is skipped 
            aucs.append(np.nan)
            start_times.append(np.nan)
            continue

        start_idx_rel = np.argmax(above)         # find exact time point, where trace crosses threshold
        start_time = time_segment[start_idx_rel]

        idx_auc_start = idx_search_start + start_idx_rel # define, where AUC window starts and ends 
        idx_auc_end = np.searchsorted(time, start_time + window_sec)

        auc_trace = trace[idx_auc_start:idx_auc_end]
        baseline_corr = auc_trace - baseline            # substracts baseline 
        auc = np.trapz(baseline_corr, dx=interval)      # compute AUC using trapezoids

        aucs.append(auc)                # safe results for this cell 
        start_times.append(start_time)

    return aucs, start_times, np.nanmean(aucs)

# Peak Amplitude for each cell -> measures peak height above a baseline in a defined time window
def analyze_peak_amplitude(data, time, baseline_start, baseline_end, 
                           peak_start, peak_end):
    #converts alle times in indizes
    idx_base_start = np.searchsorted(time, baseline_start)
    idx_base_end = np.searchsorted(time, baseline_end)
    idx_peak_start = np.searchsorted(time, peak_start)
    idx_peak_end = np.searchsorted(time, peak_end)

    amplitudes = [] # initialize list for later storage of the results

    for i in range(data.shape[1]): # for each cell
        baseline = np.min(data[idx_base_start:idx_base_end, i]) # baseline: minimum in baseline window 
        peak = data[idx_peak_start:idx_peak_end, i]
        amp = np.max(peak - baseline) # maximum in peak window - baseline
        amplitudes.append(amp)

    return amplitudes, np.nanmean(amplitudes)

# Perform Analysis (example values must be inserted)
auc1, start1, mean_auc1 = analyze_auc_per_cell(
    data, time,
    baseline_start=baseline1_start, baseline_end=baseline1_end,
    search_start=None, search_end=None,
    window_sec=window1,
    buffer=threshold_buffer
)

auc2, start2, mean_auc2 = analyze_auc_per_cell(
    data, time,
    baseline_start=baseline2_start, baseline_end=baseline2_end,
    search_start=None, search_end=None,
    window_sec=window2,
    buffer=threshold_buffer
)

amp1, mean_amp1 = analyze_peak_amplitude(data, time, 
                                         baseline_start=baseline1_start, 
                                         baseline_end=baseline1_end,
                                         peak_start=None, peak_end=None)

amp2, mean_amp2 = analyze_peak_amplitude(data, time, 
                                         baseline_start=baseline2_start, 
                                         baseline_end=baseline2_end,
                                         peak_start=None, peak_end=None)



# Export results to Excel 
df_results = pd.DataFrame({
    "Cell": [f"cell {i+1}" for i in range(num_cells)],
    "Peak Amplitude 1": amp1,
    "Peak Amplitude 2": amp2,
    "AUC Peak 1": auc1,
    "AUC Peak 2": auc2,
    "Start Time Peak 1": start1,
    "Start Time Peak 2": start2
})

# Add mean row 
df_results.loc[len(df_results)] = [
    "mean",
    mean_amp1,
    mean_amp2,
    mean_auc1,
    mean_auc2,
    np.nan,
    np.nan
]

# Export to Excel file
df_results.to_excel("Results_Threshold_Analysis.xlsx", index=False)
